In [5]:
import pandas as pd
%run preprocessing.ipynb

KeyError: 1990

KeyError: 1990

In [4]:
import pandas as pd
from datetime import datetime

# Create a dictionary to map month_id to month and year
month_id_map = {}
for year in range(1990, 2022):
    for month in range(1, 13):
        month_id = (year - 1990) * 12 + month + 120
        month_id_map[month_id] = datetime(year, month, 1)

# Convert month_id to datetime
master_df['date'] = master_df['month_id'].map(month_id_map)


master_df

NameError: name 'master_df' is not defined

In [1]:
master_df = master_df.head
master_df.tocsv("test.csv")

NameError: name 'master_df' is not defined

In [3]:
import networkx as nx
import pandas as pd

# Graph implementation assuming a no neighborhood sampling and no sliding window approach 
def create_graph(master_df):
    graph = nx.Graph()
    unique_priogrid_gids = master_df['priogrid_gid'].unique()

    for priogrid_gid in unique_priogrid_gids:
        node_features = master_df[master_df['priogrid_gid'] == priogrid_gid].drop(['priogrid_gid'], axis=1).to_dict('records')[0]
        graph.add_node(priogrid_gid, **node_features)

    # Add edges connecting each node to its 8 immediate neighbors
    for node in graph.nodes():
        row, col = node // 100, node % 100
        neighbors = []

        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                neighbor_row = row + dr
                neighbor_col = col + dc
                neighbor_priogrid_gid = neighbor_row * 100 + neighbor_col
                if neighbor_priogrid_gid in graph.nodes():
                    neighbors.append(neighbor_priogrid_gid)

        for neighbor in neighbors:
            graph.add_edge(node, neighbor)

    return graph

graph_pure = create_graph(master_df)

In [6]:

# Creating a random sliding window of size 8, selecting 4 nodes prior and 4 nodes after
import pandas as pd
import networkx as nx

G = nx.Graph()

for gid, group in master_df.groupby('priogrid_gid'):
    features = group[master_df.columns.drop('priogrid_gid')].to_dict('records')[0]
    G.add_node(gid, **features)

nodes = list(G.nodes())
for i in range(len(nodes)):
    node = nodes[i]
    neighbors = nodes[max(0, i-4):i] + nodes[i+1:i+5]
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

In [7]:
# Sliding window implementation adjacency matrix
adj_matrix_slide = nx.adjacency_matrix(G)
print(len(adj_matrix_slide.todense()))


13110


C:\Users\Divith Narendra\AppData\Local\Temp\ipykernel_6136\3921403832.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(G)


In [8]:
# Adjacency matrix for the graph without neighborhood sampling
adj_matrix = nx.adjacency_matrix(graph_pure)
print(len(adj_matrix.todense()))


13110


C:\Users\Divith Narendra\AppData\Local\Temp\ipykernel_6136\1782947322.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(graph_pure)


In [ ]:
# Implements a neighborhood sampling solution for the graph, considering a randomization factor of 0.50 
import networkx as nx
import pandas as pd
import random

def create_graph(master_df, sampling_rate=0.5):
    graph = nx.Graph()
    unique_priogrid_gids = master_df['priogrid_gid'].unique()

    for priogrid_gid in unique_priogrid_gids:
        node_features = master_df[master_df['priogrid_gid'] == priogrid_gid].drop(['priogrid_gid'], axis=1).to_dict('records')[0]
        graph.add_node(priogrid_gid, **node_features)
        
        row, col = priogrid_gid // 100, priogrid_gid % 100
        neighbors = []
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                neighbor_row = row + dr
                neighbor_col = col + dc
                neighbor_priogrid_gid = neighbor_row * 100 + neighbor_col
                if neighbor_priogrid_gid in graph.nodes():
                    neighbors.append(neighbor_priogrid_gid)

        for neighbor in neighbors:
            if random.random() <= sampling_rate:
                graph.add_edge(priogrid_gid, neighbor)

    return graph

graph = create_graph(master_df)